In [28]:
!pip install llama-cloud-services>=0.1.0

In [2]:
from typing import Dict, Any
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
# from docling.datamodel.accelerator_options import AcceleratorDevice, AcceleratorOptions
from docling.datamodel.pipeline_options import smolvlm_picture_description
from docling.document_converter import DocumentConverter, PdfFormatOption

c:\whitebox\envs\gen-prod\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_path" in LayoutModelConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\whitebox\envs\gen-prod\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_spec" in LayoutOptions has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = True  # Enable OCR for scanned documents
pipeline_options.ocr_options.use_gpu = False
pipeline_options.do_table_structure = True  # Extract table structure
pipeline_options.table_structure_options.do_cell_matching = True
# pipeline_options.do_formula_enrichment = True
# pipeline_options.generate_picture_images = True
# pipeline_options.images_scale = 2
# pipeline_options.do_picture_classification = True
# pipeline_options.do_picture_description = True
pipeline_options.picture_description_options = smolvlm_picture_description

In [4]:
converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
            }
        )

In [58]:

import fitz
from PIL import Image
pdf_path = "CHEELIZZA PIZZA INDIA LTD - INVESTMENT DECK.pdf"
doc = fitz.open(pdf_path)

In [59]:
full_text = ""
tables = []
images = []

# Process each page
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    
    # Extract text from page
    page_text = page.get_text("text")  # Specify text format explicitly
    print(page_text[:50])
    full_text += f"\n--- Page {page_num + 1} ---\n"
    full_text += page_text
    

*Private and Conﬁdential
SAY CHEESE!!
DOESN’T IT B
Current Indian 
F&B landscape
Indian Food Service 
Global Brand
Indian Brand
1990
2000
2001
2010
2011
Pizza Stores per Mn population
Phase-wise QSR Stor
QSR
Auto-2Ws
Auto-4Ws
Consumer
Durables
FMCG
Footw
VEGETARIAN FOOD PREFERENCES
%
of Vegetarian
Popula
 Current Food Service Market
Pure Veg Food Consump
Evolution of Food Business In India
When Domino’s,
Become the First Trusted “PURE VEG” Brand
Brand (k
Biryani Can – India’s First Pure-Veg Biryani Brand
 Biryani Blues
North-Indian heavy, meat-led menu
W
India’s Biryani Market: A ₹30,000–38,000 Cr 
Goldm
Built To Scale
Full-Stack Operations Driving 
Marg
Cheelizza operates as a food-tech platform, not ju
Cheeliza
Business
06
ONION
No of stores
18
MRR
 1.
Robust Organization System & Process
Customer Sati
Why Customer move to cheeliza?
Is the Cometition c
Store Unit
Economics
Apr'25 Store Operating 
profi
Growth In New Stores Opened
After Raising Seed Rou
Our Customer Loves Us Digital B

In [7]:
print(result.document.name)

CHEELIZZA PIZZA INDIA LTD - INVESTMENT DECK


In [10]:
print(result.document.origin)

mimetype='application/pdf' binary_hash=2797362585584270075 filename='CHEELIZZA PIZZA INDIA LTD - INVESTMENT DECK.pdf' uri=None


In [40]:
print(result.document.pictures[0].model_dump().keys())

dict_keys(['self_ref', 'parent', 'children', 'content_layer', 'label', 'prov', 'captions', 'references', 'footnotes', 'image', 'annotations'])


In [41]:
print(result.document.pictures[0])

self_ref='#/pictures/0' parent=RefItem(cref='#/body') children=[] content_layer=<ContentLayer.BODY: 'body'> label=<DocItemLabel.PICTURE: 'picture'> prov=[ProvenanceItem(page_no=1, bbox=BoundingBox(l=315.87042236328125, t=402.30165100097656, r=909.1106567382812, b=158.37625122070312, coord_origin=<CoordOrigin.BOTTOMLEFT: 'BOTTOMLEFT'>), charspan=(0, 0))] captions=[] references=[] footnotes=[] image=None annotations=[]


In [42]:
text = result.document.export_to_markdown()

In [44]:
full_text = result.document.export_to_markdown()

# Extract tables if any
tables = []
for table in result.document.tables:
    table_data = {
        "caption": getattr(table, 'caption', ''),
        "data": table.export_to_dataframe().to_dict('records') if hasattr(table, 'export_to_dataframe') else []
    }
    tables.append(table_data)

# Extract key-value pairs and metadata
metadata = {
    "page_count": len(result.document.pages),
    "title": getattr(result.document, 'title', ''),
    "tables_count": len(tables),
    "has_images": len(result.document.pictures) > 0,
    "word_count": len(full_text.split()) if full_text else 0
}

# Structure the extracted content
extracted_content = {
    "full_text": full_text,
    "tables": tables,
    "metadata": metadata,
}

In [1]:
filename = "CHEELIZZA PIZZA INDIA LTD - INVESTMENT DECK.pdf"

In [49]:
from jinja2 import Environment, FileSystemLoader
import os
template_dir = "./../prompts"
jinja_env = Environment(loader=FileSystemLoader(template_dir))
template = jinja_env.get_template("ipo_analysis_prompt.j2")
        
prompt = template.render(
    filename=filename,
    full_text=extracted_content.get("full_text", ""),
    # sections=content.get("sections", {}),
    tables=extracted_content.get("tables", []),
    metadata=extracted_content.get("metadata", {})
)

In [2]:
from jinja2 import Environment, FileSystemLoader
import os
template_dir = "./../prompts"
jinja_env = Environment(loader=FileSystemLoader(template_dir))
template = jinja_env.get_template("pdf_extraction_prompt.j2")
        
prompt = template.render(
    filename=filename,
    total_pages=10
)
# print(prompt)

In [3]:
import os
import instructor
from instructor import Mode
from jinja2 import Environment, FileSystemLoader
from google.genai import Client, types
import google.generativeai as genai

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise ValueError("GEMINI_API_KEY environment variable is required")

genai_client = Client(api_key=api_key)
    
# Initialize Gemini model with Instructor
model = instructor.from_genai(
    client=genai_client,
    mode=Mode.GENAI_STRUCTURED_OUTPUTS
)

In [6]:
from instructor.multimodal import PDF

In [7]:

file = PDF.from_path(filename)
print(file)

source=WindowsPath('CHEELIZZA PIZZA INDIA LTD - INVESTMENT DECK.pdf') media_type='application/pdf'


In [8]:
messages = [
    {
        "role": "user",
        "content": prompt
        
    },
    {
        "role": "user",
        "content": file
    }
]

In [21]:
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from datetime import datetime

class ExtractedTable(BaseModel):
    """Model for extracted table data"""
    title: str = Field(..., description="Title or caption of the table")
    headers: List[str] = Field(..., description="Column headers")
    rows: List[List[str]] = Field(..., description="Table rows data")
    page_number: int = Field(..., description="Page number where table was found")
    context: str = Field(..., description="Context or description around the table")

class ExtractedImage(BaseModel):
    """Model for extracted image information"""
    description: str = Field(..., description="Description of the image content")
    type: str = Field(..., description="Type of image (chart, graph, logo, diagram, etc.)")
    page_number: int = Field(..., description="Page number where image was found")
    key_insights: List[str] = Field(..., description="Key insights from the image")
    data_points: Dict[str, Any] = Field(..., description="Extracted data points if applicable")

class ExtractedSection(BaseModel):
    """Model for extracted document sections"""
    title: str = Field(..., description="Section title")
    content: str = Field(..., description="Section content")
    page_number: int = Field(..., description="Page number where section starts")
    key_points: List[str] = Field(..., description="Key points from this section")

class FinancialMetrics(BaseModel):
    """Model for extracted financial metrics"""
    revenue: str = Field(..., description="Revenue figures mentioned/ 'N/A' if not mentioned")
    profit: str = Field(..., description="Profit/loss figures/ 'N/A' if not mentioned")
    growth_rate: str = Field(..., description="Growth rate percentages/ 'N/A' if not mentioned")
    funding_raised: str = Field(..., description="Funding amounts raised/ 'N/A' if not mentioned")
    valuation: str = Field(..., description="Company valuation/ 'N/A' if not mentioned")
    burn_rate: str = Field(..., description="Monthly/yearly burn rate/ 'N/A' if not mentioned")
    runway: str = Field(..., description="Financial runway/ 'N/A' if not mentioned")
    other_metrics: str = Field(..., description="Other financial metrics found/ 'N/A' if not mentioned")

class CompanyInfo(BaseModel):
    """Model for extracted company information"""
    company_name: str = Field(..., description="Company name/ 'N/A' if not mentioned")
    industry: str = Field(..., description="Industry or sector/ 'N/A' if not mentioned")
    founded_year: int = Field(..., description="Year company was founded/ 'N/A' if not mentioned")
    location: str = Field(..., description="Company location/headquarters/ 'N/A' if not mentioned")
    stage: str = Field(..., description="Business stage (startup, growth, etc.)/ 'N/A' if not mentioned")
    employees: str = Field(..., description="Number of employees/ 'N/A' if not mentioned")
    website: str = Field(..., description="Company website/ 'N/A' if not mentioned")
    description: str = Field(..., description="Company description/mission/ 'N/A' if not mentioned")

class MarketInfo(BaseModel):
    """Model for extracted market information"""
    market_size: str = Field(..., description="Total addressable market size/ 'N/A' if not mentioned")
    target_market: str = Field(..., description="Target market description/ 'N/A' if not mentioned")
    market_opportunity: str = Field(..., description="Market opportunity description/ 'N/A' if not mentioned")
    competitors: List[str] = Field(..., description="List of competitors mentioned")
    competitive_advantage: str = Field(..., description="Competitive advantages/ 'N/A' if not mentioned")
    customer_segments: List[str] = Field(..., description="Customer segments")

class TeamInfo(BaseModel):
    """Model for extracted team information"""
    founders: List[str] = Field(..., description="Founder names and backgrounds")
    key_team_members: List[str] = Field(..., description="Key team members")
    advisors: List[str] = Field(..., description="Advisors and board members")
    team_size: str = Field(..., description="Total team size/ 'N/A' if not mentioned")
    key_hires: List[str] = Field(..., description="Key hires or positions to fill")

class BusinessModel(BaseModel):
    """Model for extracted business model information"""
    revenue_model: str = Field(..., description="How the company makes money/ 'N/A' if not mentioned")
    pricing_strategy: str = Field(..., description="Pricing strategy/ 'N/A' if not mentioned")
    customer_acquisition: str = Field(..., description="Customer acquisition strategy/ 'N/A' if not mentioned")
    distribution_channels: List[str] = Field(..., description="Distribution channels")
    partnerships: List[str] = Field(..., description="Key partnerships")
    scalability: str = Field(..., description="Business scalability factors/ 'N/A' if not mentioned")

class TractionMetrics(BaseModel):
    """Model for extracted traction metrics"""
    customers: str = Field(..., description="Number of customers/ 'N/A' if not mentioned")
    users: str = Field(..., description="Number of users/ 'N/A' if not mentioned")
    revenue_growth: str = Field(..., description="Revenue growth metrics/ 'N/A' if not mentioned")
    user_growth: str = Field(..., description="User growth metrics/ 'N/A' if not mentioned")
    retention_rate: str = Field(..., description="Customer retention rate/ 'N/A' if not mentioned")
    key_metrics: str = Field(..., description="Other key traction metrics/ 'N/A' if not mentioned")
    milestones: List[str] = Field(..., description="Key milestones achieved")

class RiskFactors(BaseModel):
    """Model for extracted risk factors"""
    market_risks: List[str] = Field(..., description="Market-related risks")
    operational_risks: List[str] = Field(..., description="Operational risks")
    financial_risks: List[str] = Field(..., description="Financial risks")
    regulatory_risks: List[str] = Field(..., description="Regulatory risks")
    competitive_risks: List[str] = Field(..., description="Competitive risks")
    technology_risks: List[str] = Field(..., description="Technology risks")
    other_risks: List[str] = Field(..., description="Other identified risks")

class FundingInfo(BaseModel):
    """Model for extracted funding information"""
    funding_amount: str = Field(..., description="Amount of funding being raised/ 'N/A' if not mentioned")
    use_of_funds: List[str] = Field(..., description="How funds will be used")
    previous_funding: List[str] = Field(..., description="Previous funding rounds")
    investors: List[str] = Field(..., description="Current/previous investors")
    equity_offered: str = Field(..., description="Equity percentage offered/ 'N/A' if not mentioned")
    valuation: str = Field(..., description="Pre/post money valuation/ 'N/A' if not mentioned")

class PDFExtractionResult(BaseModel):
    """Complete PDF extraction result model"""
    # Document structure
    full_text: str = Field(..., description="Complete extracted text from all pages")
    sections: List[ExtractedSection] = Field(..., description="Identified sections")
    
    # Business information
    company_info: CompanyInfo = Field(..., description="Company information")
    market_info: MarketInfo = Field(..., description="Market information")
    team_info: TeamInfo = Field(..., description="Team information")
    business_model: BusinessModel = Field(..., description="Business model")
    
    # Financial and traction data
    financial_metrics: FinancialMetrics = Field(..., description="Financial metrics")
    traction_metrics: TractionMetrics = Field(..., description="Traction metrics")
    funding_info: FundingInfo = Field(..., description="Funding information")
    
    # Risk assessment
    risk_factors: RiskFactors = Field(..., description="Identified risk factors")
    
    # Additional insights
    recommendations: List[str] = Field(..., description="Recommendations for improving the pitch deck")

In [22]:
response = model.create(
                messages=messages,
                response_model=PDFExtractionResult,
                model="gemini-2.5-flash",
                config=types.GenerateContentConfig(
                    temperature=0.1,  # Low temperature for more consistent extraction
                    thinking_config=types.ThinkingConfig(thinking_budget=2000)
                ),
                max_retries=3
            )

In [25]:
from pprint import pprint

In [26]:
pprint(response.model_dump())

{'business_model': {'customer_acquisition': 'N/A',
                    'distribution_channels': [],
                    'partnerships': [],
                    'pricing_strategy': 'N/A',
                    'revenue_model': 'N/A',
                    'scalability': 'N/A'},
 'company_info': {'company_name': 'CHEELIZZA PIZZA INDIA LTD',
                  'description': 'N/A',
                  'employees': 'N/A',
                  'founded_year': 2015,
                  'industry': 'Food & Beverage - Pizza Restaurants',
                  'location': 'India',
                  'stage': 'Growth Stage (Pre-IPO)',
                  'website': 'N/A'},
 'financial_metrics': {'burn_rate': 'N/A',
                       'funding_raised': 'N/A',
                       'growth_rate': 'N/A',
                       'other_metrics': 'N/A',
                       'profit': 'N/A',
                       'revenue': 'N/A',
                       'runway': 'N/A',
                       'valuation': 'N/A'},

In [27]:
from llama_cloud_services import LlamaParse

ModuleNotFoundError: No module named 'llama_cloud_services'

In [ ]:
llama_api_key = os.getenv("LLAMA_CLOUD_API_KEY")
if not llama_api_key:
    raise ValueError("LLAMA_CLOUD_API_KEY environment variable is required")

# Initialize LlamaParse for PDF text extraction
llama_parser = LlamaParse(
    api_key=llama_api_key,
    num_workers=4,
    verbose=True,
    language="en"
)